**数字化转型的测度**：
无形资产投资法/吴非法/袁淳法/赵辰宇法/胡元林法/csmar/其他
**企业绩效的测度**
主业绩效：MRS = (营业利润－投资收益－公允价值变动收益＋对联营企业和合营企业的投资收益)/ 总资产
托宾Q值
净资产收益率（ROE）
总资产收益率（ROA）
基于熵权法拟合如下指标：净资产收益率、资本积累率、资产负债率、总资产周转率

**营运绩效**
固定资产周转率/应收账款周转率/应付账款周转率/总资产周转率/存货周转率

控制变量：
董事会规模/独董比例/企业资产/两职合一/股权集中度/上市年限/营业收入/产权性质
资产负债率/资本密集度/

可能的中介变量
技术创新：创新投入，创新产出 : 研发投入；研发投入占比；科研人员数量；科研人员数量占比；
管理创新：成本利润
供应链集成
内部控制

研究思路：数字化通过技术创新 提升企业营运绩效/数字化转型通过营运绩效影响企业盈利绩效

数字化转型可以分为三个阶段

如何将数字化转型测度问题分阶段实现？数字化三个阶段——数据上传，数据分析

In [1]:
# 财务绩效数据整理
import pandas as pd
from utils import financial_dict
import warnings
warnings.filterwarnings('ignore')

trans_dict={'A001000000':'资产总计', 'B001302000':'投资收益', 'B001302101':'对联营企业和合营企业的投资收益', 'B001301000':'公允价值变动收益', 'B001300000':'营业利润','LISTINGDATE':'上市日期','PROVINCE':'上市省份','LISTINGSTATE':'上市状态','IndustryCodeC':'行业代码'}
paths=['C:/datas/企业绩效数据/其他数据/FS_Combas.txt','C:/datas/企业绩效数据/其他数据/FS_Comins.txt','C:/datas/企业绩效数据/其他数据/STK_LISTEDCOINFOANL.txt']

MRS = pd.read_csv(paths[0],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
MRS = MRS[(MRS.Typrep == "A") & (MRS.Accper.dt.month == 12)& (MRS.Accper.dt.year > 2010)][['Stkcd','Accper','A001000000']]
MRS_ = pd.read_csv(paths[1],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
MRS_ = MRS_[(MRS_.Typrep == "A") & (MRS_.Accper.dt.month == 12)& (MRS_.Accper.dt.year > 2010)][['Stkcd','Accper','B001302000','B001302101','B001301000','B001300000']]
MRS = pd.merge(MRS, MRS_, how='outer',on=['Stkcd','Accper'])

MRS_ = pd.read_csv(paths[2],sep='\t', header=0, dtype={'Symbol': 'object'}, parse_dates=['EndDate','LISTINGDATE'])
MRS_ = MRS_[(MRS_.EndDate.dt.month == 12)& (MRS_.EndDate.dt.year>2010)][['Symbol','EndDate','LISTINGDATE','PROVINCE','LISTINGSTATE','IndustryCodeC']]
MRS_.rename(columns={'Symbol':'Stkcd','EndDate':'Accper'},inplace=True)
MRS = pd.merge(MRS, MRS_, how='outer',on=['Stkcd','Accper'])
MRS.rename(columns=trans_dict,inplace=True)
# MRS = (营业利润－投资收益－公允价值变动收益＋对联营企业和合营企业的投资收益)/ 总资产
MRS['主业绩效'] = MRS['营业利润']- MRS['投资收益'] - MRS['对联营企业和合营企业的投资收益'] -MRS['公允价值变动收益']
MRS['主业绩效']  =MRS['主业绩效']/MRS['资产总计']
MRS = MRS[['Stkcd','Accper','上市日期','上市状态','上市省份','行业代码','主业绩效','资产总计']]
paths=[
    r'C:/datas/企业绩效数据/财务绩效数据/盈利能力195507730(仅供盐城工学院使用)/FI_T5.txt',
    r'C:/datas/企业绩效数据/财务绩效数据/发展能力195614784(仅供盐城工学院使用)/FI_T8.txt',
    r'C:/datas/企业绩效数据/财务绩效数据/偿债能力195205880(仅供盐城工学院使用)/FI_T1.txt',
    r'C:/datas/企业绩效数据/财务绩效数据/相对价值指标195621915(仅供盐城工学院使用)/FI_T10.txt',
    r'C:/datas/企业绩效数据/财务绩效数据/经营能力195458346(仅供盐城工学院使用)/FI_T4.txt'
]
profitability_names = [financial_dict[name] for name in ['股票代码','截止日期','总资产净利润率(ROA)B','净资产收益率（ROE）B','投入资本回报率（ROIC）','研发费用率','成本费用利润率','销售期间费用率']]
profitability = pd.read_csv(paths[0],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
profitability = profitability[(profitability.Typrep == "A") & (profitability.Accper.dt.month == 12)& (profitability.Accper.dt.year > 2010)][profitability_names]
development_names = [financial_dict[name] for name in ['股票代码','截止日期','资本积累率B']]
development = pd.read_csv(paths[1],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
development = development[(development.Typrep == "A") & (development.Accper.dt.month == 12)& (development.Accper.dt.year > 2010)][development_names]
debt_names=[financial_dict[name] for name in ['股票代码','截止日期','资产负债率']]
debt = pd.read_csv(paths[2],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
debt = debt[(debt.Typrep == "A") & (debt.Accper.dt.month == 12)& (debt.Accper.dt.year > 2010)][debt_names]
relative_names=[financial_dict[name] for name in ['股票代码','截止日期','托宾Q值B']]
relative = pd.read_csv(paths[3],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
relative = relative[(relative.Accper.dt.month == 12)& (relative.Accper.dt.year > 2010)][relative_names]
business_names=[financial_dict[name] for name in ['股票代码','截止日期','应收账款周转率B','应付账款周转率B','存货周转率B','固定资产周转率B','总资产周转率B','资本密集度']]
business = pd.read_csv(paths[4],sep='\t', header=0, dtype={'Stkcd': 'object'}, parse_dates=['Accper'])
business = business[(business.Typrep == "A") & (business.Accper.dt.month == 12)& (business.Accper.dt.year > 2010)][business_names]

business = pd.merge(profitability,business,on=['Stkcd','Accper'],how='outer')
business = pd.merge(development,business,on=['Stkcd','Accper'],how='outer')
business = pd.merge(debt,business,on=['Stkcd','Accper'],how='outer')
business = pd.merge(relative,business,on=['Stkcd','Accper'],how='outer')
business = pd.merge(MRS,business,on=['Stkcd','Accper'],how='outer')
# financial_dict
trans_dict = {value: key for key, value in financial_dict.items()}
business.rename(columns=trans_dict,inplace=True)
business['截止日期'] = business['截止日期'].dt.year
business['截止日期'] =business['截止日期'].astype('int')
business['上市状态'] = business['上市状态'].apply(lambda x:1 if x=='正常上市' else 0)

非财务绩效
控制变量：
董事会规模/独董比例/两职合一/股权集中度/两权分离度/股权性质

营业收入 = 主营业务收入
企业资产 = ln(资产总计)|ln(主营业务收入)

上市年限
技术创新：创新投入 研发投入占比；科研人员数量占比；

管理创新：成本利润
供应链集成
内部控制

In [2]:
dataset = pd.read_csv('./data/非财务绩效数据.csv',dtype={'股票代码':'object'})
dataset['供应链集成'] = (dataset['本期销售额占年度销售总额比例(%)']+dataset['本期采购额占年度采购总额比例(%)'])/2
dataset.drop(['财务波动','长期绩效增长','授权专利数2','授权专利数','纳税总额A','捐赠总额','职工薪酬','研发人员数量','员工人数','研发投入金额','本期采购额占年度采购总额比例(%)','本期销售额占年度销售总额比例(%)','股权性质'],axis=1,inplace=True)
dataset = dataset[dataset['截止年份']>2010]
dataset.rename(columns={'截止年份':'截止日期'}, inplace=True)
dataset = pd.merge(dataset, business, on=['股票代码','截止日期'], how='outer')

# 数字化情况
digital_path = r'C:/onedrivers/OneDrive - jsnu.edu.cn/projects/digital/02.数字化转型水平测度与评价/datas/数字化转型程度2022(未筛选).csv'
digital = pd.read_csv(digital_path,dtype={'股票代码':'object'})


dataset= pd.merge(dataset, digital, on=['股票代码','截止日期'], how='outer')
starttime = dataset[['股票代码','上市日期']].dropna().drop_duplicates('股票代码')
starttime['上市日期'] = starttime['上市日期'].dt.year.astype(int)
provence = dataset[['股票代码','上市省份']].dropna().drop_duplicates('股票代码')
dataset.drop(['statue','所属省份','上市日期','上市省份'],axis=1,inplace=True)
dataset = pd.merge(dataset,starttime,on='股票代码',how='outer')
dataset = pd.merge(dataset,provence,on='股票代码',how='outer')
dataset

,截止日期,股票代码,董事会规模,两职合一,主营业务收入,两权分离度(%),股权集中度4(%),研发人员数量占比(%),研发营收比(%),内部控制指数,...,资本密集度,wufei,zhaochenyv,yuanchun,无形资产数字化,csmar,lishouxi,股权性质,上市日期,上市省份
0,2011,000001,272946.0,1.0,NaN,NaN,58.9121,NaN,NaN,716.87,...,43.255160,0.000000,0.000019,1.320859,0.000299,0.360857,0.000039,0.0,1991.0,广东省
1,2012,000001,263477.0,1.0,NaN,NaN,58.7076,NaN,NaN,724.92,...,41.287048,0.693147,0.000220,0.597921,0.000344,0.373089,0.128662,0.0,1991.0,广东省
2,2013,000001,298070.0,1.0,NaN,NaN,74.7385,NaN,NaN,773.70,...,36.880356,2.890372,0.000338,0.789940,0.000224,0.467492,0.627915,0.0,1991.0,广东省
3,2014,000001,275838.0,1.0,NaN,NaN,64.5121,NaN,NaN,590.73,...,30.953452,3.496508,0.000701,1.189093,0.000247,0.449356,0.852747,0.0,1991.0,广东省
4,2015,000001,332918.0,1.0,NaN,NaN,66.0596,NaN,NaN,620.15,...,27.044377,3.526361,0.000852,1.600131,0.000198,0.470867,1.370381,0.0,1991.0,广东省
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53049,2018,900957,24833.0,1.0,6.470736e+07,13.0328,48.0287,NaN,NaN,NaN,...,9.489895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,上海市
53050,2019,900957,23928.0,1.0,1.050223e+08,13.0328,48.1793,NaN,NaN,NaN,...,9.690460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,上海市
53051,2020,900957,23191.0,1.0,NaN,13.0328,48.3093,NaN,NaN,NaN,...,10.145928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,上海市
53052,2021,900957,22442.0,1.0,NaN,13.0328,48.3658,NaN,NaN,NaN,...,9.052161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,上海市


In [3]:
# 创新产出 :
import numpy as np
from utils import fillna__

grant = pd.read_excel(r'C:/datas/企业绩效数据/非财务绩效数据/专利授权情况表(1992-2017)142534725(仅供盐城工学院使用)/上市公司专利申请和授权数据.xlsx', dtype={'股票代码':'object'})
grant = grant[(grant['公司类型']=='集团公司合计')&(grant['year']>2010)]
grant['独立申请实用新型&外观设计'] = grant['当年独立申请的实用新型数量']  + grant['当年独立申请的外观设计数量']
grant['联合申请实用新型&外观设计'] = grant['当年联合申请的实用新型数量']  + grant['当年联合申请的外观设计数量']
grant['独立获得实用新型&外观设计'] = grant['当年独立获得的实用新型数量']  + grant['当年独立获得的外观设计数量']
grant['联合获得实用新型&外观设计'] = grant['当年联合获得的实用新型数量']  + grant['当年联合获得的外观设计数量']
grant.drop(['stkcd','公司类型','当年独立申请的实用新型数量','当年独立申请的外观设计数量','当年联合申请的实用新型数量','当年联合申请的外观设计数量','当年独立获得的外观设计数量','当年独立获得的实用新型数量','当年联合获得的外观设计数量','当年联合获得的实用新型数量'], axis=1, inplace=True)
grant.rename(columns={'year':'截止日期'}, inplace=True)
dataset= pd.merge(dataset, grant, on=['股票代码','截止日期'], how='outer')
dataset.sort_values(by =['股票代码','截止日期'],inplace=True)
dataset.reset_index(inplace=True,drop=True)
grants=['当年独立申请的发明数量', '当年联合申请的发明数量', '当年独立获得的发明数量', '当年联合获得的发明数量', '当年独立申请专利总和', '当年联合申请专利总和', '当年独立获得专利总和', '当年联合获得专利总和', '独立申请实用新型&外观设计', '联合申请实用新型&外观设计', '独立获得实用新型&外观设计', '联合获得实用新型&外观设计']
for name in grants:
    dataset[name] = dataset[name].apply(lambda x: np.log(1+x))
dataset = dataset[(dataset['截止日期']<2023)&(2010<dataset['截止日期'])]
dataset['两职合一'].fillna(1,inplace=True)
dataset['股权性质'].fillna(0,inplace=True)
dataset['上市状态'].fillna(0,inplace=True)
dataset['上市日期'] = dataset['上市日期'].astype(str)
# 获取所有曾经被St的企业
sts = set(dataset[dataset['上市状态']==0]['股票代码'].tolist())
with open('./data/stcodes.txt','w',encoding='utf-8') as fp:
    fp.write('\n'.join(sts))
dataset = fillna__(dataset,'股票代码', rate = 0.2)
dataset['上市日期'] = dataset['上市日期'].astype(float)
dataset.dropna(subset=['上市日期','行业代码'], inplace=True)
dataset['上市期限'] = dataset['截止日期']-dataset['上市日期']
dataset['企业规模'] =dataset['资产总计'].apply(lambda x: np.log(1+x))
dataset = dataset[dataset['上市状态']==1]
dataset.drop(['上市日期','资产总计','上市状态'],axis=1,inplace=True)
dataset.sort_values(['股票代码','截止日期'], inplace=True)
dataset.to_csv('./data/因果机制研究全部数据.csv',encoding='utf8',index=False)

创新绩效(Innovation):
有些学者直接使用 发明专利的申请数来衡量企业的创新绩效，专利数量是研究中常见的用于测量企业创新产出的指标。发明专利的创新价值最高，实用新型专利、外观设计专利某种程度上存在策略性创新的倾向，发明专利则更能体现企业创新的“质变”特征。
专利授权容易受到来自官僚因素的影响，且专利授权往往要经历较长的审核周期，具有一定的不确定性和不稳定性(黎文靖和郑曼妮,2016)。
专利申请数量更能反映企业的资源投入力度和使用效率。
鉴于一些企业的专利申请数量可能为0,本文对相应的数据采取加1取对数方式构造被解释变量。

[1]	李雪松, 党琳, 赵宸宇. 数字化转型、融入全球创新网络与创新绩效[J]. 中国工业经济, 2022(10): 43-61.
有些学者选择将创新绩效划分为
利用式创新：研发费用化支出取自然对数/发明专利申请数
探索式创新：研发资本化支出取自然对数/实用新型及外观专利申请数